## LAB 5
 ### Name: Guru Charan Bogireddy  Student ID: 8902043

 #### Here I attached 4 the graphs and generated html page to show case my work

In [1]:
#import libraries
from sklearn import datasets
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np
from typing import List
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [2]:
#import dataset
X, y = datasets.load_diabetes(as_frame=True, scaled=False, return_X_y=True)

# split the dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Multiple Polynomial Models
Let's create a function to automate this process:

In [3]:
def create_polynomial_models(X: np.ndarray, y: np.ndarray, degrees: List[int]) -> dict[int, np.poly1d]:
    """
    Creates polynomial models for the given degrees and fits them to the given data.

    Args
        X: The x values of the data.
        y: The y values of the data.
        degrees: A list of polynomial degrees to try.

    Returns:
        A dictionary of polynomial models, with the polynomial degree as the key.
    """

    models = {}
    for degree in degrees:
        model = Pipeline([('polynomial', PolynomialFeatures(degree=degree)),
                          ('linear', LinearRegression())])
        model.fit(X, y)
        models[degree] = model
    return models

With the function that we've just defined, let's create 9 polynomial models, from degree 0 (constant) and up to a degree of 5:

In [21]:
degrees = list(range(0, 9))
#models = create_polynomial_models(X.reshape(-1, 1), y, degrees)
models = create_polynomial_models(X, y, degrees)

In [7]:
X

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,59.0,2.0,32.1,101.00,157.0,93.2,38.0,4.00,4.8598,87.0
1,48.0,1.0,21.6,87.00,183.0,103.2,70.0,3.00,3.8918,69.0
2,72.0,2.0,30.5,93.00,156.0,93.6,41.0,4.00,4.6728,85.0
3,24.0,1.0,25.3,84.00,198.0,131.4,40.0,5.00,4.8903,89.0
4,50.0,1.0,23.0,101.00,192.0,125.4,52.0,4.00,4.2905,80.0
...,...,...,...,...,...,...,...,...,...,...
437,60.0,2.0,28.2,112.00,185.0,113.8,42.0,4.00,4.9836,93.0
438,47.0,2.0,24.9,75.00,225.0,166.0,42.0,5.00,4.4427,102.0
439,60.0,2.0,24.9,99.67,162.0,106.6,43.0,3.77,4.1271,95.0
440,36.0,1.0,30.0,95.00,201.0,125.2,42.0,4.79,5.1299,85.0


In [20]:
cross_val_scores = []
modelScores =[["Degree","Rscore","RSDscore", "mseScore", "maeScore","maeSDScore"]]

for degree in degrees:
    # Create a pipeline that creates polynomial features, then applies linear regression
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    
    # Use cross-validation to compute the mean score across multiple splits of the data
    rsqscore = cross_val_score(model, X, y, cv=5).mean()
    rsqSDScore = cross_val_score(model, X, y,  cv=5).std()
    mseScore = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    maeScore = cross_val_score(model, X, y,  cv=5, scoring="neg_mean_absolute_error").mean()
    maeSDScore = cross_val_score(model, X, y,  cv=5, scoring="neg_mean_absolute_error").std()

    cross_val_scores.append(rsqscore)
    modelScores.append([degree, rsqscore, rsqSDScore, mseScore,maeScore,maeSDScore])

# Choose the degree with the highest cross-validation score
best_degree = degrees[np.argmax(cross_val_scores)]
print(f"The best degree is {best_degree}")


The best degree is 1


In [22]:
dfModelScores = pd.DataFrame(modelScores)
dfModelScores

,0,1,2,3,4,5
0,Degree,Rscore,RSDscore,mseScore,maeScore,maeSDScore
1,0,-0.027506,0.036772,-5982.413414,-66.045624,3.47466
2,1,0.482316,0.049269,-2993.08131,-44.276499,2.10011
3,2,-0.347201,0.646338,-8202.528724,-63.546355,14.94552
4,3,-203.419848,225.878682,-1168778.934326,-342.052617,142.438549
5,4,-571.083108,369.891883,-3402771.880899,-657.260477,159.475902
6,5,-436.856887,379.100423,-2644546.971482,-562.993636,59.917202
7,6,-1695.47736,2632.416214,-10572837.707301,-742.560741,191.043026
8,7,-5530.894074,9518.586953,-34758833.967742,-1032.681725,393.439617
9,8,-16076.255118,28049.952581,-101245028.538799,-1475.658532,706.280274


3. Identification of the Best Model: Identify the model that exhibits the highest performance based on the R-Squared and MAE metrics. Provide an explanation for choosing this specific model.

After model trained for 9 degrees, the **degree one** is the best model from the all of above as it has higher or R^2, MAE and MSE.